<a href="https://colab.research.google.com/github/WoradeeKongthong/medical_cost_regression/blob/master/09_MedicalCost_ANN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np
import pandas as pd

# Dataset

In [0]:
df = pd.read_csv('https://raw.githubusercontent.com/WoradeeKongthong/medical_cost_regression/master/cleaned_insurance.csv', usecols=[0,1,2,3,4,5,6])

In [3]:
df

,age,sex,bmi,children,smoker,region,charges
0,19,female,27.900,0,True,southwest,16884.92400
1,18,male,33.770,1,False,southeast,1725.55230
2,28,male,33.000,3,False,southeast,4449.46200
3,33,male,22.705,0,False,northwest,21984.47061
4,32,male,28.880,0,False,northwest,3866.85520
...,...,...,...,...,...,...,...
1333,50,male,30.970,3,False,northwest,10600.54830
1334,18,female,31.920,0,False,northeast,2205.98080
1335,18,female,36.850,0,False,southeast,1629.83350
1336,21,female,25.800,0,False,southwest,2007.94500


# Data Preprocessing

## X and y

In [0]:
X = df.iloc[:,:-1].values
y = df.iloc[:, -1].values

In [5]:
X

array([[19, 'female', 27.9, 0, True, 'southwest'],
       [18, 'male', 33.77, 1, False, 'southeast'],
       [28, 'male', 33.0, 3, False, 'southeast'],
       ...,
       [18, 'female', 36.85, 0, False, 'southeast'],
       [21, 'female', 25.8, 0, False, 'southwest'],
       [61, 'female', 29.07, 0, True, 'northwest']], dtype=object)

In [6]:
y

array([16884.924 ,  1725.5523,  4449.462 , ...,  1629.8335,  2007.945 ,
       29141.3603])

## Encoding categorical data

In [0]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder

In [0]:
ct = ColumnTransformer(transformers=[('encoder', OneHotEncoder(drop='first'),[1,4,5])],remainder='passthrough')

In [0]:
X = ct.fit_transform(X)

In [10]:
X[:5]

array([[0.0, 1.0, 0.0, 0.0, 1.0, 19, 27.9, 0],
       [1.0, 0.0, 0.0, 1.0, 0.0, 18, 33.77, 1],
       [1.0, 0.0, 0.0, 1.0, 0.0, 28, 33.0, 3],
       [1.0, 0.0, 1.0, 0.0, 0.0, 33, 22.705, 0],
       [1.0, 0.0, 1.0, 0.0, 0.0, 32, 28.88, 0]], dtype=object)

## Train-test split

In [0]:
from sklearn.model_selection import train_test_split

In [0]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

# ANN

In [0]:
from keras.models import Sequential
from keras.layers import Dense

In [49]:
X_train.shape

(1070, 8)

## Create the ANN

In [0]:
regressor = Sequential()
regressor.add(Dense(units=15,kernel_initializer='normal',activation='relu',input_dim=8))
regressor.add(Dense(units=10,kernel_initializer='normal',activation='relu'))
regressor.add(Dense(units=10,kernel_initializer='normal',activation='relu'))
regressor.add(Dense(units=5,kernel_initializer='normal',activation='relu'))
regressor.add(Dense(units=1,kernel_initializer='normal'))

## Compile the created ANN

In [51]:
regressor.compile(optimizer='adam', loss='mean_squared_error')
regressor.summary()

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_24 (Dense)             (None, 15)                135       
_________________________________________________________________
dense_25 (Dense)             (None, 10)                160       
_________________________________________________________________
dense_26 (Dense)             (None, 10)                110       
_________________________________________________________________
dense_27 (Dense)             (None, 5)                 55        
_________________________________________________________________
dense_28 (Dense)             (None, 1)                 6         
Total params: 466
Trainable params: 466
Non-trainable params: 0
_________________________________________________________________


## Train the ANN

In [52]:
regressor.fit(X_train, y_train, batch_size=10, epochs=1000)

Epoch 1/1000
1070/1070 [==============================] - 0s 203us/step - loss: 316855330.0187
Epoch 2/1000
1070/1070 [==============================] - 0s 116us/step - loss: 312761608.0374
Epoch 3/1000
1070/1070 [==============================] - 0s 110us/step - loss: 254478691.8785
Epoch 4/1000
1070/1070 [==============================] - 0s 116us/step - loss: 145178886.8785
Epoch 5/1000
1070/1070 [==============================] - 0s 120us/step - loss: 129655508.7290
Epoch 6/1000
1070/1070 [==============================] - 0s 115us/step - loss: 129064486.8411
Epoch 7/1000
1070/1070 [==============================] - 0s 114us/step - loss: 128966058.8411
Epoch 8/1000
1070/1070 [==============================] - 0s 111us/step - loss: 128776628.0748
Epoch 9/1000
1070/1070 [==============================] - 0s 114us/step - loss: 128453688.1682
Epoch 10/1000
1070/1070 [==============================] - 0s 116us/step - loss: 128336027.1776
Epoch 11/1000
1070/1070 [========================

## Prediction

In [0]:
y_pred = regressor.predict(X_test)

Print sample of predictions

In [55]:
np.set_printoptions(precision=2)
print('y_test     y_pred')
print(np.concatenate((y_test.reshape(len(y_test),1),y_pred.reshape(len(y_pred),1)),axis=1)[:10])

y_test     y_pred
[[16069.08 16215.4 ]
 [ 8891.14 10307.26]
 [42856.84 39827.8 ]
 [ 9411.   11228.46]
 [ 8516.83 10017.38]
 [ 1712.23  1957.64]
 [ 9863.47 10702.01]
 [11743.93 12488.99]
 [15828.82  9341.77]
 [19798.05 19611.01]]


Evaluation

In [59]:
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from math import sqrt
mae = mean_absolute_error(y_test,y_pred)
mse = mean_squared_error(y_test,y_pred)
rmse = sqrt(mse)
r2 = r2_score(y_test,y_pred)

n = X_test.shape[0]
p = X_test.shape[1]
adj_r2 = 1 - (1 - r2) * ((n - 1) / (n - p - 1)) 

print('MAE = ',mae)
print('MSE = ',mse)
print('RMSE = ',rmse)
print('R-squared = ',r2)
print('Adj R-squared = ',adj_r2)

MAE =  2726.818586980294
MSE =  22976268.386177946
RMSE =  4793.356692984358
R-squared =  0.8465950870399553
Adj R-squared =  0.8418567113500698


Note :  
- the model we create needs small batch size  
- the model needs large number of epochs